# TestCase3 - Convergence FEM / PINNs+Corr

In [ ]:
from create_jsonfile import ask_user,read_config

# ask_user()
config = read_config()

In [2]:
dimension = config["dimension"]
testcase = config["testcase"]
version = config["version"]
if testcase == 3:
    largenet = False
    if "largenet" in version:
        largenet = True
    version = "medium"
param_num = config["param_num"]        

In [3]:
high_degree = 10

## Imports + Load

In [4]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd
import os
import dolfin as df
import seaborn as sns
import dataframe_image as dfi

# from modfenics.fenics_expressions.fenics_expressions_2D import UexExpr
from scimba.equations.domain import SpaceTensor
from scimba.equations import domain

from testcases.utils import create_tree
# from modfenics.fenics_expressions.fenics_expressions import FExpr,AnisotropyExpr
# from modfenics.utils import get_param,compute_slope
from modfenics.error_estimations.fem import compute_error_estimations_fem_deg,compute_error_estimations_fem_all
from modfenics.error_estimations.add import compute_error_estimations_Corr_deg,compute_error_estimations_Corr_all,plot_Corr_vs_FEM
from modfenics.error_estimations.mult import compute_error_estimations_Mult_deg,compute_error_estimations_Mult_all,plot_Mult_vs_FEM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
assert dimension == 2
if dimension == 2:
    result_dir = "../../../results/fenics/test_2D/testcase"+str(testcase)+"/version"+str(version)
    if testcase == 1:
        if version == 1:
            from testcases.networks.test_2D.test_1.test_1_v1 import Run_laplacian2D,Poisson_2D
        elif version == 2:
            from testcases.networks.test_2D.test_1.test_1_v2 import Run_laplacian2D,Poisson_2D
        else:
            raise ValueError("Version not found")
        from testcases.problem.problem_2D import TestCase1
        problem = TestCase1(version=version)
    elif testcase == 2:
        from testcases.networks.test_2D.test_2.test_2_v1 import Run_laplacian2D,Poisson_2D
        from testcases.problem.problem_2D import TestCase2
        problem = TestCase2(version=version)
    elif testcase == 3:
        if version != "new":
            from testcases.networks.test_2D.test_3.test_3_v1 import Run_laplacian2D,Poisson_2D
        else:
            from testcases.networks.test_2D.test_3.test_3_v2 import Run_laplacian2D,Poisson_2D
        from testcases.problem.problem_2D import TestCase3
        problem = TestCase3(version=version)
        result_dir = "../../../results/fenics/test_2D/testcase"+str(testcase)+"/"+version
        if version == "medium" and largenet:
            result_dir += "_largenet"
    elif testcase == 4:
        from testcases.networks.test_2D.test_4.test_4_v1 import Run_laplacian2D,Poisson_2D
        from testcases.problem.problem_2D import TestCase4
        problem = TestCase4(v=version)

result_dir += "/cvg/"
create_tree(result_dir)
    # elif testcase == 4:

In [ ]:
if dimension == 2:
    if testcase == 3 and version != "new":
        pde = Poisson_2D(version)
        trainer,u_theta = Run_laplacian2D(pde,version,largenet)
    else:
        pde = Poisson_2D()
        trainer,u_theta = Run_laplacian2D(pde)

## Convergence FEM

In [8]:
# degree = 2
# df_FEM, tab_nb_vert_FEM, tab_h_FEM, tab_err_FEM = compute_error_estimations_fem_deg(param_num,problem,degree,high_degree,new_run=False,result_dir=result_dir)

In [ ]:
compute_error_estimations_fem_all(param_num,problem,high_degree,new_run=False,result_dir=result_dir,plot_cvg=True)

## Convergence PINNs+Corr

In [23]:
# degree = 2
# df_Corr, tab_nb_vert_Corr, tab_h_Corr, tab_err_Corr = compute_error_estimations_Corr_deg(param_num,problem,degree,high_degree,u_theta,new_run=False,result_dir=result_dir)

In [ ]:
compute_error_estimations_Corr_all(param_num,problem,high_degree,u_theta,new_run=False,result_dir=result_dir,plot_cvg=True)

### Plots FEM-Corr

In [ ]:
plot_Corr_vs_FEM(param_num,problem,result_dir=result_dir)

## Convergence PINNs+Mult

In [13]:
# M=0.0
# degree = 2
# df_Corr, tab_nb_vert_Corr, tab_h_Corr, tab_err_Corr = compute_error_estimations_Mult_deg(param_num,problem,degree,high_degree,u_theta,M=M,new_run=False,result_dir=result_dir)

In [ ]:
tab_M = [0.0,0.1,1.0,100.0]

for M in tab_M:
    print("#### M = ",M)
    compute_error_estimations_Mult_all(param_num,problem,high_degree,u_theta,M=M,new_run=False,result_dir=result_dir,plot_cvg=True)